In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import numpy as np
import tensorflow as tf
import pathlib
from tqdm import trange

from src.DataPreprocessor.data_preprocessor import DataPreprocessor, Mode
from src.LearningKeras.net_architecture import cnn_150x150x5,cnn_150x150x5_3class
from src.LearningKeras.train import KerasTrainer

np.random.seed(1)
tf.set_random_seed(2)

check data_visualisation for details

In [2]:
from src.DataPreprocessor.DataIOBackend.gdal_backend import GdalBackend
from src.DataPreprocessor.data_visualiser import DataVisualiser

dataiobackend = GdalBackend()
data_preprocessor = DataPreprocessor(data_dir="data/Region 1 - Lopukangri/",
                              backend=dataiobackend,
                              filename_prefix="tibet",
                              mode=Mode.TRAIN,
                              seed=1)

## NN

### Train

In [3]:
model_generator = lambda: cnn_150x150x5_3class()
ensemble_size = 3
batch_size = 10

trainer = KerasTrainer(model_generator=model_generator,
                       ensemble_size=ensemble_size,
                       data_preprocessor=data_preprocessor,
                       batch_size=batch_size)
train_generator = data_preprocessor.train_generator(batch_size=batch_size,
                                         class_probabilities=np.array([1./3, 1./3, 1./3]),
                                         patch_size=(150, 150),
                                         channels=np.array([0, 1, 2, 3, 4]))

In [4]:
sampl = next(train_generator)
print(sampl[0].shape)
print(sampl[1].shape)

(10, 150, 150, 5)
(10, 3)


In [5]:
history_arr = trainer.train(steps_per_epoch=2, epochs=5, train_generator=train_generator)

Epoch 1/5
2/2 [==============================] - 11s 5s/step - loss: 1.3396 - acc: 0.1500 - val_loss: 1.7591 - val_acc: 0.3200
Epoch 2/5
2/2 [==============================] - 9s 4s/step - loss: 1.7364 - acc: 0.2500 - val_loss: 1.0419 - val_acc: 0.4400
Epoch 3/5
1/2 [==============>...............] - ETA: 2s - loss: 1.0596 - acc: 0.6000

KeyboardInterrupt: 

In [ ]:
for i in range(ensemble_size):
    pathlib.Path('models_3class').mkdir(parents=True, exist_ok=True)
    trainer.models[i].save_weights('models_3class/model_{}.h5'.format(i))